Defining constants and calculation functions

In [ ]:
import math
from typing import Generator

directions = {"U": 0, "R": 1, "D": 2, "L": 3}

def sequence_generation(string: str, max_len=8) -> Generator[str]:
    yield string
    if len(string) < max_len:
        for valid_dir in {d for d in directions.keys() if directions[d] <= len(string)}:
            yield from sequence_generation(string + valid_dir, max_len)

def pattern_entropy(string: str):
    entropy = 0
    for char in directions.keys():
        count = string.count(char)
        if count == 0: continue
        char_probability = float(count) / len(string)
        entropy -= char_probability * math.log(char_probability, 2)
    return entropy

def lempel_ziv_stats(string):
    encoding_dict = {}
    for direction in directions.keys():
        if direction in string:
            encoding_dict[direction] = str(len(encoding_dict))

    working_string = ""
    output_codes = []
    for c in string:
        if working_string + c in encoding_dict:
            working_string += c
        else:
            output_codes.append(encoding_dict[working_string])
            encoding_dict[working_string + c] = str(len(encoding_dict))
            working_string = c

    output_codes.append(encoding_dict[working_string])
    return len(output_codes), len(encoding_dict)


Let's also define the "strength" of each challenge the player will have to go through. A rough analogue would be a level cap in a nuzlocke

In [ ]:
level_caps = [1.5 + (i * 0.1) + 0.3 * (i ** 1.8) for i in range(10)]
level_caps

Start with all unique sequences up to some maximum length

In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 300)

max_sequence_length = 12

sequences = np.fromiter(sequence_generation("U", max_sequence_length), "U16", count=-1)

sequences_with_complexity = pd.DataFrame(sequences, columns=["Input pattern"])
just_sequences = sequences_with_complexity["Input pattern"]

Compute columns for length and entropy

In [ ]:
vectorized_length = np.vectorize(len)
sequences_with_complexity["length"] = vectorized_length(just_sequences)

vectorized_entropy = np.vectorize(pattern_entropy)
sequences_with_complexity["entropy"] = vectorized_entropy(just_sequences)

Compute and concatenate columns for compression encoding stats

In [ ]:
vectorized_lzw = np.vectorize(lempel_ziv_stats)
sequences_with_complexity = pd.concat(
    [
        sequences_with_complexity,
        pd.DataFrame(np.array(vectorized_lzw(just_sequences)).T, columns=["compressed length", "encoding dict size"]),
    ], axis=1
)

sequences_with_complexity

Scatter length vs entropy

In [ ]:
import matplotlib.pyplot as plt

points = sequences_with_complexity[["length", "entropy"]].drop_duplicates()

fig, ax = plt.subplots()
ax.scatter(points["length"], points["entropy"])

plt.show()

Scatter length vs encoded length

In [ ]:
length_and_compressed_length = np.column_stack((sequences_with_complexity["length"], sequences_with_complexity["compressed length"]))
lengths_T = length_and_compressed_length.T
length_and_compressed_length = list(zip(lengths_T[0], lengths_T[1]))

unique, counts = np.unique(length_and_compressed_length, axis=0, return_counts=True)
unique = [(length, encoded_len) for length, encoded_len in unique]

count_dict = dict(zip(unique, counts))

def lookup(r):
    return count_dict[(*r,)]

big_dot_sizes = np.apply_along_axis(lookup, arr=sequences_with_complexity[["length", "compressed length"]].to_numpy(), axis=1)

In [ ]:

sequences_with_complexity["scaled dot sizes"] = big_dot_sizes / (count_dict[max(count_dict, key=count_dict.get)] / 10) + 1
sequences_with_complexity

In [ ]:
points = sequences_with_complexity[["length", "compressed length", "scaled dot sizes"]].drop_duplicates()

fig, ax = plt.subplots()
ax.scatter(x=points["length"],
           y=points["compressed length"],
           s=points["scaled dot sizes"],
           )

plt.show()

And now for dict size

In [ ]:


fix, ax = plt.subplots()
ax.scatter(x=sequences_with_complexity["length"],
           y=sequences_with_complexity["encoding dict size"],
           )

plt.show()

See if there's large deviations between encoded length and encoding dict size

In [ ]:
fix, ax = plt.subplots()
ax.scatter(sequences_with_complexity["compressed length"], sequences_with_complexity["encoding dict size"])

plt.show()

Checking entropy vs encoding size

In [ ]:
fix, ax = plt.subplots()
points = sequences_with_complexity[["entropy", "compressed length"]].drop_duplicates()
ax.scatter(y=points["entropy"], x=points["compressed length"])

plt.show()

Trying out a formula, aiming for worst pattern to grow by 0.1 with length, and best pattern to grow quadratically.

`RR` is 1.1, `RU` is 1.2.

`RRR` is 1.2, `RUL` is 1.5

`RRRR` is 1.3, `RULD` is 2

Idea: linear "base" per length, then scale entropy based on the difference between length and compressed length

In [ ]:
# shorter name for conciseness
seq = sequences_with_complexity

seq["complexity"] = (((seq["length"] - 1) * 0.1 + 1)
                     * (seq["entropy"]
                        * (seq["length"]
                           / (1 + seq["length"] - seq["compressed length"])
                           )
                        )
                     )

Graphing lengths vs complexity to see the spread

In [ ]:
points = seq[["Input pattern", "length", "complexity", "entropy"]].drop_duplicates(subset=["length", "complexity"])

fix, ax = plt.subplots()
ax.scatter(x=points["length"],
           y=points["complexity"],
           s=(points["entropy"] * 10),)

plt.show()

That's an interesting gap, what's causing it?

In [ ]:
twelve_length = points[points["length"] == 12]
twelve_length_below_gap = twelve_length[twelve_length["complexity"] < 30]
twelve_length_above_gap = twelve_length[twelve_length["complexity"] >= 30]
highest_below_gap = twelve_length_below_gap[twelve_length_below_gap["complexity"] == max(twelve_length_below_gap["complexity"])]
lowest_above_gap = twelve_length_above_gap[twelve_length_above_gap["complexity"] == min(twelve_length_above_gap["complexity"])]
print(highest_below_gap)
print(lowest_above_gap)

In [ ]:
print(seq[seq["Input pattern"] == "URRRUDULDDLL"])
print(seq[seq["Input pattern"] == "URRUUDRDDULU"])

It looks like being able to compress the pattern just by one symbol is a big jump. Do the maximum complexity patterns have the shape we want?

In [ ]:
print(seq[seq["complexity"] == max(seq["complexity"])])

Too many repeated letters in each sequence. Let's add a metric to account for those

In [ ]:
def count_repeated_letters(s: str) -> int:
    count = 0
    for previous, current in zip(s, s[1:]):
        if previous == current: count += 1
    return count

seq["repeated letters"] = seq["Input pattern"].apply(count_repeated_letters)

seq[["Input pattern", "repeated letters"]]

Setting the "compressability multiplier" to be equal to $$\log_{10}({\text{length} \over 1 + \text{length} - \text{compressed length}})$$

In [ ]:
from math import log10

seq["base length multiplier"] = (seq["length"] - 1) * 0.1 + 1
seq["compressability multiplier"] = (seq["length"] / (1 + seq["length"] - seq["compressed length"])).apply(lambda x: log10(9 + (max(1, x))))
seq[["Input pattern", "length", "compressability multiplier"]].sort_values("compressability multiplier")

In [ ]:
# the lowest the bonus could be (asymptotically approaches). 1 is good, can be less than to reduce the chance any bonus is given for many repeats
lower_bound = 0.97
# How fast the maximum bonus scales based on length. This is an exponent, should probably keep between 1 and 2
max_bonus_scaling = 1.3
# At what length should the no-repeats bonus be 2
length_for_double = 18
# from 0 to 1, where 1 has the whole string as a repeated input, where half of the bonus should be received
proportion_for_half_bonus = 0.2
# steepness of the curve. Higher values make the curve steeper, and flatten the asymptotic approach on either side of the midpoint. Must be greater than 0
steepness = 1.25
# Additional parameter for affecting character of the curve. > 0
skew = 1.5

upper_bound = 1 + ((seq["length"] / length_for_double) ** max_bonus_scaling)

mid_shift_inside_ln = (((upper_bound - lower_bound) ** skew) / ((((upper_bound + lower_bound) / 2) - lower_bound) ** skew)) - 1

mid_shift = seq["length"] * proportion_for_half_bonus - (1.0 / steepness) * mid_shift_inside_ln.apply(np.log)

exponent = steepness * (seq["repeated letters"] - mid_shift)
bottom_of_frac = (1 + exponent.apply(np.exp)) ** (1 / skew)
seq["no repeats bonus"] = (lower_bound + ((upper_bound - lower_bound) / bottom_of_frac)).apply(lambda x: max(1, x))
seq[["length", "repeated letters", "compressability multiplier", "no repeats bonus"]].sort_values("no repeats bonus")

Initial values look alright, let's see the graph

In [ ]:
points = seq[["length", "no repeats bonus"]].drop_duplicates()

fix, ax = plt.subplots()
ax.scatter(x=points["length"],
           y=points["no repeats bonus"],)
ax.grid('on')

plt.show()

Very good spread. I like that it appears 12 with 5 repeated inputs gives the same bonus as 4 with 1. Let's calculate the new, improved complexity from this

In [ ]:
compressability_factor = 0.5
no_repeats_factor = 0.8
entropy_factor = 0.5

seq["improved complexity"] = (seq["base length multiplier"]
                              * (((((seq["compressability multiplier"] - 1) * compressability_factor) + 1)
                              * (((seq["no repeats bonus"] - 1) * no_repeats_factor) + 1)
                              * (1 + (seq["entropy"] * entropy_factor / 2))) ** 1.3)).round(2)



In [ ]:
fix, ax = plt.subplots()
points = pd.concat([seq["length"], seq["improved complexity"]], axis=1).drop_duplicates()

ax.scatter(points["length"], points["improved complexity"])
ax.grid('on')
plt.show()

That's looking good. Decent jump from full repeats to a little bit of change, to more quickly demonstrate that complexity is key. Can this handle exponential growth?

In [ ]:
downscale_factor = 1
seq["downscaled complexity"] = ((seq["improved complexity"] - 1) / downscale_factor + 1) ** 1

fix, ax = plt.subplots()
points = pd.concat([seq["length"], seq["downscaled complexity"]], axis=1).drop_duplicates()


ax.scatter(points["length"], points["downscaled complexity"])
ax.grid('on')

# lines showing the "power level" of each level cap. The idea is that each red line should provide the same amount of challenge, so the maximum complexity available needs to be provided accordingly
# ax.hlines(list(filter(lambda x: x < max(seq["downscaled complexity"]) + 1, level_caps)) , 0, 1, transform=ax.get_yaxis_transform(), colors="r", linewidth=1)
plt.show()

Maaaaaaaaybe. Exponential grows very fast as the pattern gets large. Quadratic or cubic might be better, but we definitely want to capture the feeling of "the 15th input is harder to remember than the 5th". Still need to remember that challenges aren't going to be balanced around the max possible complexity for the max length available, but instead a linearly-increasing threshold. It should be possible to do a boss both around max-level 6-length and mid-level 9-length (which the graph here shows pretty nicely, hm)

Otherwise, looking decent. How do patterns look?

In [ ]:
percentiles = [0.0, 0.3, 0.6, 0.9, 1]

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 400)

for i in seq["length"].drop_duplicates():
    at_len = seq[seq["length"] == i][["Input pattern", "length", "entropy", "compressability multiplier", "downscaled complexity"]]
    quantiles = at_len["downscaled complexity"].quantile(percentiles, "nearest")
    print(at_len[at_len["downscaled complexity"].isin(quantiles)].drop_duplicates(["downscaled complexity"]).sort_values("downscaled complexity"))